# Preparation

In [10]:
import pandas as pd
import networkx as nx
import netwulf as nw


In [11]:
od_f = pd.read_csv("data/NYCTAXI_OD_f.csv")

# od_f["flow"].value_counts()
od_f

,origin_id,destination_id,flow,cost
0,0,0,0.0,0.0
1,0,1,0.0,31349.0
2,0,2,0.0,33747.0
3,0,3,0.0,17205.0
4,0,4,0.0,14927.0
...,...,...,...,...
69164,262,258,25.0,15740.0
69165,262,259,32.0,5206.0
69166,262,260,24.0,9612.0
69167,262,261,1634.0,571.0


# Construct network

In [12]:
G = nx.DiGraph()
for i, row in od_f.iterrows():
    if row["flow"] > 0:
        G.add_edge(row["origin_id"], row["destination_id"], flow=row["flow"])
    else:
        G.add_node(row["origin_id"])
        G.add_node(row["destination_id"])

In [13]:
# G = nx.barabasi_albert_graph(100,m=1)
# stylized_network, config = nw.visualize(G, plot_in_cell_below=False)
# fig, ax = nw.draw_netwulf(stylized_network, figsize=(10,10))
# plt.savefig("myfigure.pdf")